In [22]:
import matplotlib

%matplotlib inline
matplotlib.interactive(False)
import os
import warnings

warnings.filterwarnings("ignore")

import librosa
import librosa.display
import matplotlib.pyplot as plt
from noisereduce.noisereducev1 import reduce_noise
import sklearn
from mutagen.mp3 import MP3
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np


N_FFT = 1024  # Number of frequency bins for Fast Fourier Transform
HOP_SIZE = 1024  # Number of audio frames between STFT columns
SR = 44100  # Sampling frequency
N_MELS = 30  # Mel band parameters
WIN_SIZE = 1024  # number of samples in each STFT window
FMIN = 1500

In [12]:
import splitfolders
splitfolders.ratio('E:/train_audio', output='E:/', seed=1337, ratio=(.8, .2)) 

Copying files: 14852 files [00:15, 986.53 files/s] 


In [12]:
birds = []
dire = 'E:/bird_data/'
for root, dirs, files in os.walk('E:/train'):
    if root == "E:/train":
        birds = dirs

birds30 = []
flist_all = []

for i, bird in enumerate(birds):
    blist_call_train = []
    for root, dirs, files in os.walk('E:/train/' + bird):
        for file in files:
            if file.endswith(".ogg"):
                blist_call_train.append(os.path.join(root, file))
                
    flist_all.append(blist_call_train)
    birds30.append(bird)

print(len(flist_all[0]))

12


In [23]:
birds = []
dire = 'E:/bird_data/'
for root, dirs, files in os.walk(dire+"xeno-canto-dataset-Australia-call/"):
    if root == dire + "xeno-canto-dataset-Australia-call/":
        birds = dirs
        
birds30 = []
flist_call_train = []
flist_song_train = []
flist_all_train = []

for i, bird in enumerate(birds):
    blist_call_train = []
    for root, dirs, files in os.walk(dire + "output-xeno-canto-dataset-Australia-call/train/" + bird):
        for file in files:
            if file.endswith(".mp3"):
                blist_call_train.append(os.path.join(root, file))
    flist_call_train.append(blist_call_train)
    
    blist_song_train = []
    for root, dirs, files in os.walk(dire + "output-xeno-canto-dataset-Australia-song/train/" + bird):
        for file in files:
            if file.endswith(".mp3"):
                blist_song_train.append(os.path.join(root, file))
    flist_song_train.append(blist_song_train)
    
    flist_all_train.append(blist_call_train + blist_song_train)
    birds30.append(bird)

print(len(flist_call_train[0]), len(flist_song_train[0]), len(flist_all_train[0]))

12 23 35


In [3]:
flist_call_test = []
flist_song_test = []
flist_all_test = []

for i, bird in enumerate(birds):
    blist_call_test = []
    for root, dirs, files in os.walk(dire + "output-xeno-canto-dataset-Australia-call/val/" + bird):
        for file in files:
            if file.endswith(".mp3"):
                blist_call_test.append(os.path.join(root, file))
    flist_call_test.append(blist_call_test)
    
    blist_song_test = []
    for root, dirs, files in os.walk(dire + "output-xeno-canto-dataset-Australia-song/val/" + bird):
        for file in files:
            if file.endswith(".mp3"):
                blist_song_test.append(os.path.join(root, file))
    flist_song_test.append(blist_song_test)
    
    flist_all_test.append(blist_call_test + blist_song_test)

In [5]:
from random import sample

# select random 10 recordings
randFiles = sample(range(len(flist_call)), 10)
print('Selected random files number:',randFiles)

for audioFile in randFiles:
    data, sr = librosa.load(flist_call[audioFile][0], duration=5) # sr = sampling rate

    f, axs = plt.subplots(1,3,figsize=(18,5))
    plt.subplot(1,3,1)
    librosa.display.specshow(
        librosa.core.amplitude_to_db(
            librosa.feature.melspectrogram(
                            y=data,
                            sr=sr, 
                            n_fft=N_FFT, 
                            hop_length=HOP_SIZE, 
                            n_mels=30, 
                            htk=True, 
                            fmin=0.0, 
                            fmax=sr/2.0), 
                ref=np.max),
            sr=sr,
            hop_length=HOP_SIZE, 
            x_axis='time', 
            y_axis='mel')
    
    plt.colorbar()
    plt.title('Mel-spectrogram, no filter, nmels=30')
    
    plt.subplot(1,3,2)
    librosa.display.specshow(
                librosa.core.amplitude_to_db(
                    librosa.feature.melspectrogram(
                                    y=data, 
                                    sr=sr, 
                                    n_fft=N_FFT, 
                                    hop_length=HOP_SIZE, 
                                    n_mels=30, 
                                    htk=True, 
                                    fmin=FMIN,
                                    fmax=sr/2.0), 
                        ref=np.max),
                    sr=sr,
                    hop_length=HOP_SIZE, 
                    x_axis='time', 
                    y_axis='mel',
                    fmin=FMIN) #fmin -correct plot scale frequency parameter
    plt.title('Mel-spectrogram, high-pass filter, nmels=30')
    plt.colorbar()
    
    plt.subplot(1,3,3)
    S = librosa.feature.melspectrogram(y=data, sr=sr, fmin=FMIN, n_mels=64) #AMPLITUDE
    librosa.display.specshow(librosa.power_to_db(S**2,ref=np.max),
                              y_axis='mel', x_axis='time', fmin=FMIN) #power = S**2
    plt.title('Mel spectrogram, high-pass filter, nmels=64')
    plt.colorbar()
    plt.show()
    print('*******************************************')

NameError: name 'flist_call' is not defined

In [24]:
#https://github.com/sleekEagle/audio_processing/blob/master/mix_noise.py
import math  
def get_white_noise(signal,SNR) :
    #RMS value of signal
    RMS_s=math.sqrt(np.mean(signal**2))
    #RMS values of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    #Additive white gausian noise. Thereore mean=0
    #Because sample length is large (typically > 40000)
    #we can use the population formula for standard daviation.
    #because mean=0 STD=RMS
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, signal.shape[0])
    return noise

In [4]:
sample_audio= flist_call[0][0]
y, sr = librosa.load(sample_audio, mono=True)

plt.title("origin")
librosa.display.waveshow(y)
plt.show()

# y = librosa.effects.pitch_shift(y, sr, n_steps=4)
# plt.title("pitch shift")
# librosa.display.waveshow(y)
# plt.show()

# y_reduce_noise = reduce_noise(audio_clip=y, noise_clip=y, verbose=False)
# plt.title("noise reduce")
# librosa.display.waveshow(y)
# plt.show()

clip_length = 5
start = 0
index=1
plt.title("5s split")
while start < len(y)/sr:
    plt.subplot(4, 1, index)
    y_cut=y[start*sr: (start + clip_length)*sr]
    plt.plot(np.arange(0, len(y_cut)) / float(sr), y_cut)
    start +=clip_length
    index+=1
plt.show()

# plt.title("white noise")
# y_noise=y + get_white_noise(y, SNR=10)
# librosa.display.waveshow(y_noise)
# plt.show()

clip_length1 = 3
clip_length2 = 5
start = 0
index=1
plt.title("5s split wrap around")
while start < len(y)/sr:
    y_cut_1=y[start*sr: (start + clip_length1)*sr]
    y_cut_2=y[(start + clip_length1)*sr : (start + clip_length2)*sr]
    y_cut = np.concatenate([y_cut_2, y_cut_1])
    plt.plot(np.arange(0, len(y_cut)) / float(sr), y_cut)
    start +=clip_length
plt.show()

# overlay = 1
# start = 0
# plt.title("5s split 1s overlay")
# while start < len(y)/sr:
#     y_cut=y[start*sr: (start + clip_length)*sr]
#     plt.plot(np.arange(0, len(y_cut)) / float(sr), y_cut)
#     start += clip_length - overlay
# plt.show()

# silent = librosa.effects.split(y_reduce_noise)
# plt.title("silent")
# cuts = []
# for start, end in silent:
#     y_cut = y[start: end]
#     cuts.append(y_cut)

# print('silent')
# for y_cut in cuts:
#     plt.plot(np.arange(0, len(y_cut)) / float(sr), y_cut)
#     plt.show()    

NameError: name 'flist_call' is not defined

In [25]:
def saveMel(y, directory):

    fig = plt.figure(1, frameon=False)
    fig.set_size_inches(6, 6)
    ax = plt.Axes(fig, [0.0, 0.0, 1.0, 1.0])
    ax.set_axis_off()
    fig.add_axes(ax)
    librosa.display.specshow(
        librosa.core.amplitude_to_db(
            librosa.feature.melspectrogram(
                #dct_type=3,
                y=y,
                sr=SR,
                n_fft=N_FFT,
                hop_length=HOP_SIZE,
                n_mels=N_MELS,
                htk=True,
                fmin=FMIN, # high-pass filter freq.
                #fmin=0,
                fmax=sr / 2.0,
            ),
            ref=1.0,
        ),
        sr=SR,
        hop_length=HOP_SIZE,
    )
    fig.savefig(directory)
    fig.clear()
    ax.cla()
    plt.clf()
    plt.close("all")

In [29]:
size = {
    "desired": 5,  # [seconds]
    "minimum": 4,  # [seconds]
    "stride": 2,  # [seconds]
    "name": 5,
}

clip_length1 = 3

for bird, birdList in enumerate(flist_call_train):
    for birdnr, path in tqdm(enumerate(birdList)):
        directory = ("E:/bird_data/silent/" + str(bird) + birds30[bird][: size["name"]] + "/")
        if not os.path.exists(directory):
            os.makedirs(directory)
        if not os.path.exists(directory + path.rsplit("/", 1)[1].replace(" ", "")[:-4] + "1_1.png"):
            y, sr = librosa.load(path, mono=True)
            y, _ = librosa.effects.trim(y)
            y = reduce_noise(audio_clip=y, noise_clip=y, verbose=False)
            #y = librosa.effects.pitch_shift(y, sr, n_steps=4)
            #silent = librosa.effects.split(y)
            step = (size["desired"] - size["stride"]) * sr

            nr = 0
            for start, end in zip(
                 range(0, len(y), step), range(size["desired"] * sr, len(y), step)
            ):
                #for start, end in silent:
                nr = nr + 1
                if end - start > size["minimum"] * sr and nr < 20:
                #if end - start > size["minimum"] * sr:
                    melpath = path.rsplit("/", 1)[1]
                    print(melpath.split('\\'))
                    melpath = (directory+ melpath.split('\\')[1]+ "_"+ str(nr)+ "song-2stride.png")
                    melpath = melpath.replace('\\','/')
                    print(melpath)

    #                     y_cut_1=y[start: start + clip_length1*sr]
    #                     y_cut_2=y[start + clip_length1*sr : end]
    #                     y_cut = np.concatenate([y_cut_2, y_cut_1])
                    y_noise=y[start:end] + get_white_noise(y[start:end], SNR=10)
                    #y_noise=y_cut[start:end] + get_white_noise(y_cut[start:end], SNR=10)

                    #saveMel(y[start:end], melpath)
                    saveMel(y_noise, melpath)
        pass

['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris146133.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris146133.mp3_1silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris146133.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris146133.mp3_2silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris146133.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris146133.mp3_3silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris146133.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris146133.mp3_4silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris146133.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris146133.mp3_5silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris146133.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris146133.mp3_6silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris171970.mp3']
E:/bird_data/silent/0Acant/

['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris673211.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris673211.mp3_1silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris673211.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris673211.mp3_2silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris673211.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris673211.mp3_3silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris673211.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris673211.mp3_4silent.png
['Acanthorhynchustenuirostris', 'Acanthorhynchustenuirostris676469.mp3']
E:/bird_data/silent/0Acant/Acanthorhynchustenuirostris676469.mp3_1silent.png



['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_1silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_2silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_3silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_4silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_5silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_6silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_7silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_8sil

['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_68silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_69silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_70silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_71silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_72silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_73silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.mp3_74silent.png
['Acridotherestristis', 'Acridotherestristis102121.mp3']
E:/bird_data/silent/1Acrid/Acridotherestristis102121.m

KeyboardInterrupt: 